# Approach 1 : LULC using hand annotated data & L8 - Invariant pixels

Data is labelled using hand annotated polygons. The model is trained using invariant pixels. 

Author: Morgane Magnier (morgane.magnier@vattenfall.com)

Copyright © 2024 Magnier Morgane 

This notebook is part of a thesis project. The copyright of the thesis itself belongs to the student Morgane Magnier.  

**Rights and Intellectual Property**:  
- Vattenfall has the right to use the findings, methods, and conclusions of this thesis in its operations.  
- Any material generated within the framework of this thesis that is subject to intellectual property protection (e.g., source code, computer program, design, or invention) belongs to Vattenfall, unless otherwise agreed in writing.  

Permission is granted to view, copy, and share this notebook for **educational or personal purposes only**, provided that this notice is included in all copies.  

---


In [1]:
import ee, geemap,eemont

import ee, eemont, geemap

import sys
sys.path.append('../preprocessing/clouds')
import landsat_preprocessing
import s2_preprocessing

sys.path.append('../wetlands_detection')
import wetlands_unsupervised_clustering


try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

In [2]:
roi = ee.Geometry.Polygon([[[17.204933,60.402663],[17.204933,60.455525],[17.2645,60.455525],[17.2645,60.402663],[17.204933,60.402663]]])

## Preprocessing functions

In [3]:
training_img = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI','MNDWI','NDMI','SAVI'], 0.6, '2023-05-01', '2023-09-30').median()

In [4]:
display(training_img)

In [5]:
m = geemap.Map()
m.centerObject(roi, 13)
m.addLayer(training_img.scale(), {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.1}, 'Sentinel-2 Image')
#m

AttributeError: 'NoneType' object has no attribute 'split'

In [6]:
def add_elev_slope(image): 

    jax_dsm = ee.ImageCollection('JAXA/ALOS/AW3D30/V3_2')
    jax_elevation = jax_dsm.select('DSM')

    proj = jax_elevation.first().select(0).projection()
    slopeReprojected =  (jax_elevation.mosaic() \
                                .setDefaultProjection(proj)).resample('bicubic')

    # Reduce the collection with a median reducer.
    elevation = slopeReprojected.reduce(ee.Reducer.mean()).rename('elev')

    slope = ee.Terrain.slope(elevation).rename('slope')

    image = image.addBands(elevation).addBands(slope)

    return image

def normalize(image, roi):
    bandNames = image.bandNames()
  # Compute min and max of the image
    minDict = image.reduceRegion(
        reducer = ee.Reducer.min(),
        geometry = roi,
        scale = 10,
        maxPixels=1e9,
        bestEffort=True,
        tileScale= 16)
    
    maxDict = image.reduceRegion(
        reducer= ee.Reducer.max(),
        geometry= roi,
        scale= 10,
        maxPixels= 1e9,
        bestEffort= True,
        tileScale= 16)
    
    mins = ee.Image.constant(minDict.values(bandNames))
    maxs = ee.Image.constant(maxDict.values(bandNames))

    normalized = image.subtract(mins).divide(maxs.subtract(mins))
    return normalized.copyProperties(image)

## Choix des dates

## Labelling

In [7]:
svämängar1 =  ee.Geometry.Polygon([[[17.24081,60.433891],[17.241325,60.434484],[17.241282,60.434971],[17.240682,60.435309],[17.239995,60.434949],[17.239566,60.434632],[17.238922,60.434441],[17.238622,60.434187],[17.23845,60.433721],[17.238622,60.43334],[17.23948,60.433277],[17.239866,60.433573],[17.240467,60.433488],[17.240767,60.433234],[17.240124,60.432832],[17.24021,60.432451],[17.240939,60.432366],[17.241368,60.432811],[17.241154,60.433298],[17.24081,60.433891]]])
svämängar2 =  ee.Geometry.Polygon([[[17.206886,60.407516],[17.2071,60.407877],[17.207572,60.407919],[17.207959,60.407686],[17.208195,60.407993],[17.20871,60.408099],[17.209461,60.40794],[17.209976,60.407771],[17.210212,60.407919],[17.209396,60.408269],[17.208838,60.408481],[17.208474,60.40865],[17.208989,60.408979],[17.209418,60.408809],[17.210062,60.408481],[17.210813,60.408279],[17.211714,60.408142],[17.212143,60.408322],[17.211649,60.408523],[17.211757,60.408629],[17.212551,60.408491],[17.21298,60.408565],[17.212915,60.408798],[17.212937,60.409074],[17.212508,60.409296],[17.212594,60.409477],[17.212636,60.409752],[17.212014,60.4099],[17.2118,60.410239],[17.212443,60.410536],[17.213452,60.410314],[17.213902,60.410187],[17.214589,60.410261],[17.214375,60.409996],[17.213516,60.409922],[17.214181,60.409699],[17.214632,60.409445],[17.214911,60.409508],[17.215426,60.409752],[17.214632,60.409106],[17.214139,60.408512],[17.213581,60.408142],[17.213538,60.407877],[17.213151,60.40758],[17.213151,60.407304],[17.213473,60.407135],[17.212336,60.407008],[17.211735,60.407209],[17.211435,60.407315],[17.211199,60.407347],[17.210662,60.407389],[17.209911,60.407315],[17.209139,60.407421],[17.208753,60.407326],[17.208581,60.407177],[17.208345,60.407336],[17.207959,60.407442],[17.206886,60.407516]]])
svämängar3 =  ee.Geometry.Polygon([[[17.219825,60.43082],[17.220726,60.430841],[17.220984,60.431434],[17.220383,60.431688],[17.220511,60.432345],[17.220984,60.432641],[17.221799,60.432133],[17.222528,60.431942],[17.221756,60.431265],[17.221541,60.430799],[17.222314,60.430672],[17.223258,60.430672],[17.224116,60.430672],[17.224202,60.430291],[17.223473,60.430121],[17.222743,60.429571],[17.222443,60.429126],[17.223902,60.42902],[17.224288,60.428766],[17.224202,60.427897],[17.223687,60.427559],[17.223988,60.426923],[17.22343,60.426436],[17.223859,60.426012],[17.224889,60.426415],[17.225618,60.426457],[17.225361,60.425949],[17.224846,60.425568],[17.224545,60.42525],[17.223902,60.425165],[17.223601,60.424212],[17.223644,60.423365],[17.223516,60.42275],[17.222786,60.422899],[17.222271,60.422602],[17.222056,60.423089],[17.221627,60.423047],[17.221885,60.423513],[17.222185,60.423704],[17.222099,60.424466],[17.223043,60.425165],[17.222528,60.42525],[17.220984,60.424043],[17.220683,60.424318],[17.221112,60.42472],[17.221498,60.425059],[17.221498,60.425462],[17.222228,60.426012],[17.222314,60.426267],[17.222056,60.42669],[17.221413,60.426711],[17.221885,60.427156],[17.222013,60.427453],[17.221456,60.427643],[17.221842,60.428003],[17.221584,60.428385],[17.220855,60.428745],[17.220254,60.429486],[17.221198,60.429571],[17.220941,60.430121],[17.220683,60.430524],[17.219825,60.43082]]])

two_union = svämängar1.union(right=svämängar2, maxError=1)
svämängar = two_union.union(right=svämängar3, maxError=1)

water1 =  ee.Geometry.Polygon([[[17.217207,60.436093],[17.215748,60.436093],[17.21549,60.435098],[17.215447,60.433552],[17.215533,60.431985],[17.215834,60.431095],[17.215576,60.430185],[17.215834,60.428914],[17.216392,60.427114],[17.216005,60.425631],[17.219868,60.426245],[17.219138,60.428596],[17.218451,60.431032],[17.218838,60.433509],[17.217851,60.434229],[17.217207,60.436093]]])
water2 =  ee.Geometry.Polygon([[[17.234802,60.437035],[17.233129,60.437427],[17.231433,60.437533],[17.229438,60.43839],[17.228193,60.438994],[17.226799,60.439502],[17.225533,60.439428],[17.225876,60.438814],[17.227056,60.438708],[17.228193,60.43839],[17.229052,60.437734],[17.229781,60.437088],[17.229631,60.436622],[17.229116,60.4364],[17.229438,60.436082],[17.230253,60.436442],[17.231348,60.436622],[17.232807,60.436336],[17.234094,60.436082],[17.235296,60.436188],[17.234802,60.437035]]])
water3 =  ee.Geometry.Polygon([[[17.254071,60.451673],[17.248149,60.449134],[17.246175,60.445937],[17.250423,60.445218],[17.256045,60.445768],[17.256775,60.447631],[17.255144,60.450552],[17.254071,60.451673]]])
w_union = water1.union(right=water2, maxError=1)
water = w_union.union(right=water3, maxError=1)

tree1 =  ee.Geometry.Polygon([[[17.225189,60.43694],[17.225425,60.437321],[17.22579,60.437639],[17.225726,60.438168],[17.225404,60.438263],[17.224803,60.43802],[17.224009,60.437639],[17.223773,60.437099],[17.223344,60.436728],[17.222936,60.436993],[17.222679,60.437395],[17.221498,60.437236],[17.220941,60.437384],[17.219524,60.437035],[17.219138,60.436453],[17.218451,60.435796],[17.218559,60.43532],[17.220168,60.434896],[17.221177,60.434854],[17.222593,60.435024],[17.22446,60.434918],[17.225039,60.435362],[17.226799,60.435458],[17.227464,60.435532],[17.228043,60.436506],[17.228022,60.437152],[17.227528,60.437437],[17.226691,60.437014],[17.226648,60.436548],[17.22609,60.436823],[17.225339,60.436516],[17.224803,60.436559],[17.225189,60.43694]]])
tree2 =  ee.Geometry.Polygon([[[17.233665,60.434716],[17.233193,60.434812],[17.232699,60.434208],[17.232206,60.434388],[17.232099,60.434769],[17.231712,60.434918],[17.231991,60.435415],[17.231648,60.435828],[17.231455,60.435892],[17.230468,60.436082],[17.229009,60.435341],[17.227228,60.434198],[17.226455,60.433954],[17.224331,60.434113],[17.222314,60.434102],[17.220941,60.433361],[17.219975,60.433097],[17.220705,60.432239],[17.221327,60.432091],[17.221971,60.431508],[17.221541,60.431032],[17.222099,60.430831],[17.222807,60.430735],[17.224116,60.430884],[17.22388,60.430386],[17.222893,60.429719],[17.222657,60.429211],[17.223966,60.428925],[17.224631,60.427516],[17.223902,60.426288],[17.225189,60.426711],[17.226391,60.426394],[17.226648,60.426965],[17.226949,60.427813],[17.22755,60.428279],[17.228279,60.429316],[17.227249,60.429719],[17.227292,60.430312],[17.228279,60.431244],[17.229996,60.432154],[17.231154,60.431222],[17.2333,60.43262],[17.233665,60.434716]]])
tree3 =  ee.Geometry.Polygon([[[17.237291,60.438782],[17.235875,60.439459],[17.234888,60.44056],[17.233729,60.440645],[17.233043,60.440285],[17.233,60.439735],[17.23227,60.439459],[17.231369,60.439142],[17.23094,60.438824],[17.230811,60.438634],[17.232485,60.437956],[17.234845,60.437511],[17.235575,60.437681],[17.235231,60.438337],[17.236047,60.43838],[17.236905,60.438147],[17.237291,60.438782]]])
tree4 =  ee.Geometry.Polygon([[[17.214031,60.423195],[17.214117,60.423704],[17.213945,60.424106],[17.213259,60.423894],[17.2124,60.424106],[17.212529,60.424466],[17.211542,60.424869],[17.210727,60.424424],[17.21004,60.423831],[17.209353,60.424339],[17.20901,60.425271],[17.208624,60.425673],[17.207851,60.425631],[17.206607,60.425186],[17.205877,60.42561],[17.205362,60.425906],[17.20459,60.426055],[17.203646,60.425928],[17.202873,60.425462],[17.205663,60.424699],[17.20562,60.42436],[17.204976,60.422899],[17.203217,60.421691],[17.201028,60.419975],[17.198496,60.420547],[17.196651,60.421077],[17.196994,60.419636],[17.199011,60.418535],[17.197852,60.41756],[17.20356,60.418344],[17.203732,60.42006],[17.205577,60.420971],[17.207508,60.421395],[17.208624,60.420441],[17.206049,60.419276],[17.207594,60.418492],[17.209911,60.418153],[17.211843,60.418492],[17.212873,60.419001],[17.213259,60.420293],[17.213945,60.422221],[17.214031,60.423195]]])
t_union = tree1.union(right=tree2, maxError=1)
t2_union = t_union.union(right=tree3, maxError=1)
tree = t2_union.union(right=tree4, maxError=1)

built1 =  ee.Geometry.Polygon([[[17.224374,60.439777],[17.223945,60.440243],[17.223301,60.440179],[17.223859,60.440793],[17.224159,60.441344],[17.224116,60.442],[17.224245,60.442423],[17.224631,60.442868],[17.224674,60.443439],[17.223988,60.443863],[17.225447,60.444053],[17.225189,60.441936],[17.224975,60.440772],[17.224374,60.439777]]])
built2 =  ee.Geometry.Polygon([[[17.227163,60.442656],[17.227979,60.441809],[17.228665,60.441449],[17.22888,60.440666],[17.229567,60.44056],[17.230554,60.440708],[17.230039,60.441195],[17.230425,60.441005],[17.231798,60.440814],[17.232656,60.441132],[17.232056,60.441449],[17.230854,60.441979],[17.229309,60.44255],[17.227979,60.442889],[17.227163,60.442656]]])
built3 =  ee.Geometry.Polygon([[[17.228794,60.38884],[17.228537,60.389858],[17.228279,60.39113],[17.227592,60.391681],[17.225618,60.391512],[17.225361,60.390367],[17.223129,60.39024],[17.222271,60.389816],[17.218838,60.389816],[17.219954,60.386465],[17.221756,60.386762],[17.221842,60.387695],[17.222872,60.387313],[17.224159,60.388162],[17.224846,60.388883],[17.228794,60.38884]]])
built4 =  ee.Geometry.Polygon([[[17.243471,60.39007],[17.245445,60.392487],[17.2439,60.39342],[17.242785,60.393674],[17.241669,60.393462],[17.240982,60.391978],[17.240982,60.391257],[17.23815,60.391427],[17.236776,60.391978],[17.234888,60.389264],[17.234545,60.389816],[17.234974,60.390876],[17.232485,60.391766],[17.230339,60.391469],[17.229738,60.389264],[17.228193,60.388798],[17.227764,60.387313],[17.230768,60.38655],[17.231712,60.387398],[17.233858,60.387568],[17.233772,60.386168],[17.232914,60.385362],[17.229738,60.385362],[17.227163,60.383029],[17.22888,60.382181],[17.227163,60.381333],[17.227678,60.380696],[17.229309,60.380781],[17.230597,60.381375],[17.232571,60.381248],[17.233171,60.380654],[17.235231,60.381205],[17.234201,60.381969],[17.236519,60.382181],[17.23772,60.38252],[17.240295,60.382096],[17.242613,60.384896],[17.243128,60.386253],[17.244673,60.386889],[17.245531,60.38778],[17.244072,60.388034],[17.243471,60.39007]]])
b_union = built1.union(right=built2, maxError=1)
b2_union = b_union.union(right=built3, maxError=1)
built = b2_union.union(right=built4, maxError=1)

bare1 =  ee.Geometry.Polygon([[[17.203517,60.433446],[17.201071,60.432514],[17.200341,60.433065],[17.199955,60.433891],[17.201285,60.434208],[17.20283,60.434484],[17.203217,60.434695],[17.203989,60.434124],[17.203517,60.433446]]])
bare2 =  ee.Geometry.Polygon([[[17.235918,60.431604],[17.234974,60.429952],[17.232656,60.428427],[17.232656,60.427664],[17.234116,60.427834],[17.234459,60.426224],[17.234373,60.425546],[17.236261,60.426139],[17.235918,60.427241],[17.236261,60.429062],[17.236347,60.429952],[17.235918,60.431604]]])
bare = bare1.union(right=bare2, maxError=1)


In [8]:
display(bare)

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "Geometry.union",
    "arguments": {
      "left": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    17.203517,
                    60.433446
                  ],
                  [
                    17.201071,
                    60.432514
                  ],
                  [
                    17.200341,
                    60.433065
                  ],
                  [
                    17.199955,
                    60.433891
                  ],
                  [
                    17.201285,
                    60.434208
                  ],
                  [
                    17.20283,
                    60.434484
                  ],
                  [
                    17.203217,
                    60.434695
                  ],
                  [
                    17.203989,
                    60.434124
                  ],
                  [
                    17.203517,
                    60.433446
                  ]
                ]
              ]
            },
            "evenOdd": {
              "constantValue": true
            }
          }
        }
      },
      "maxError": {
        "functionInvocationValue": {
          "functionName": "ErrorMargin",
          "arguments": {
            "value": {
              "constantValue": 1
            }
          }
        }
      },
      "right": {
        "functionInvocationValue": {
          "functionName": "GeometryConstructors.Polygon",
          "arguments": {
            "coordinates": {
              "constantValue": [
                [
                  [
                    17.235918,
                    60.431604
                  ],
                  [
                    17.234974,
                    60.429952
                  ],
                  [
                    17.232656,
                    60.428427
                  ],
                  [
                    17.232656,
                    60.427664
                  ],
                  [
                    17.234116,
                    60.427834
                  ],
                  [
                    17.234459,
                    60.426224
                  ],
                  [
                    17.234373,
                    60.425546
                  ],
                  [
                    17.236261,
                    60.426139
                  ],
                  [
                    17.235918,
                    60.427241
                  ],
                  [
                    17.236261,
                    60.429062
                  ],
                  [
                    17.236347,
                    60.429952
                  ],
                  [
                    17.235918,
                    60.431604
                  ]
                ]
              ]
            },
            "evenOdd": {
              "constantValue": true
            }
          }
        }
      }
    }
  }
})

### Create feature collection for training

In [13]:
polygons = ee.FeatureCollection([
  ee.Feature(svämängar, {'class': 0}),
  ee.Feature(bare, {'class': 1}),
  ee.Feature(water, {'class': 2}),
  ee.Feature(tree, {'class': 3}),
  ee.Feature(built, {'class': 4}),
])

In [14]:
display(polygons)

### Display annotated data

In [15]:
palette = ['#91cf60', '#fc8d59', '#4575b4', '#1a9850','#d73027','#fee08b']

def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

def add_color(feature):
    class_value = feature.get('class')
    color = ee.List(palette).get(class_value)
    return feature.set({'style': {'color': color, 'width': 2}})

styled_polygons = polygons.map(add_color)

polygons_image = styled_polygons.style(**{
    'styleProperty': 'style',
    'neighborhood': 8,
})

m = geemap.Map()
m.centerObject(roi, 13)
m.addLayer(apply_scale_factors(training_img).clip(roi), {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.1}, 'Sentinel-2 Image')
m.addLayer(polygons_image, {}, 'Polygons')

# Ajouter une légende à la carte
legend_dict = {
    'Northern Alluvial Meadows': '#91cf60',
    'Bare Land': '#fc8d59',
    'Water': '#4575b4',
    'Trees': '#1a9850',
    'Built': '#d73027',
    'Crops': '#fee08b'
}

m.add_legend(title="Legend", legend_dict=legend_dict)

# Afficher la carte
m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

## Training

In [16]:
def Random_Forest(composite, polygons): 

        # Get the values for all pixels in each polygon in the training.
    training_data = composite.sampleRegions(**{
    # Get the sample from the polygons FeatureCollection.
    'collection': polygons,
    # Keep this list of properties from the polygons.
    'properties': ['class'],
    # Set the scale to get Landsat pixels in the polygons.
    'scale': 10
    })

    # Training & validation splitting
    sample = training_data.randomColumn()
    train_sample = sample.filter('random <= 0.8')
    val_sample = sample.filter('random > 0.8')

    clsfr_rf = ee.Classifier.smileRandomForest(numberOfTrees = 500, variablesPerSplit = 2, minLeafPopulation = 1, bagFraction = 0.5, maxNodes = 7, seed = 11)

    trained_clsfr_rf = clsfr_rf.train(
        features=train_sample,
        classProperty='class',
        inputProperties=composite.bandNames(),)


    # Get information about the trained classifier.
    #display('Results of trained classifier RF', trained_clsfr_rf.explain())

    # Get a confusion matrix and overall accuracy for the training sample.
    train_accuracy_rf = trained_clsfr_rf.confusionMatrix()

    display('Training error matrix', train_accuracy_rf)
    display('Training overall accuracy', train_accuracy_rf.accuracy())

    # Get a confusion matrix and overall accuracy for the validation sample.
    val_sample_rf = val_sample.classify(trained_clsfr_rf)

    val_accuracy_rf = val_sample_rf.errorMatrix('class', 'classification')

    display('Validation error matrix', val_accuracy_rf)
    display('Validation accuracy', val_accuracy_rf.accuracy())

    return trained_clsfr_rf

In [19]:
classifier = Random_Forest(ee.Image(normalize(add_elev_slope(training_img),roi)), polygons)

'Training error matrix'

'Training overall accuracy'

'Validation error matrix'

'Validation accuracy'

In [1]:
val_accuracy_rf.kappa()

NameError: name 'val_accuracy_rf' is not defined

## Prediction 

### Create composite

In [ ]:
def create_annual_composites_may_to_september(collection):
    def add_year_month(image):
        date = ee.Date(image.get('system:time_start'))
        year = date.get('year')
        month = date.get('month')
        return image.set('year', year).set('month', month)
    
    # Ajouter les propriétés 'year' et 'month' à chaque image
    collection = collection.map(add_year_month)
    
    # Obtenir la liste des années uniques
    years = ee.List(collection.aggregate_array('year')).distinct().sort()
    
    # Fonction pour créer les composites annuels en utilisant uniquement les mois de mai à septembre
    def composite_year(year):
        year = ee.Number(year)
        filtered = collection.filter(ee.Filter.calendarRange(year, year, 'year'))\
                             .filter(ee.Filter.calendarRange(5, 9, 'month'))
        return filtered.median().set('year', year).set('system:time_start', ee.Date.fromYMD(year, 5, 1))

    # Créer les composites pour chaque année
    composites = years.map(lambda year: composite_year(year))

    # Retourner la collection de composites
    return ee.ImageCollection(composites)

def filter_non_empty_bands(collection):
    def has_bands(image):
        # Vérifier si l'image a des bandes en comptant les noms des bandes
        band_count = image.bandNames().size()
        return image.set('band_count', band_count)
    
    # Ajouter la propriété 'has_bands' à chaque image
    collection_with_band_info = collection.map(has_bands)
    
    # Filtrer les images qui contiennent au moins une bande
    filtered_collection = collection_with_band_info.filter(ee.Filter.gt('band_count', 0))
    
    return filtered_collection


In [ ]:
l8 = landsat_preprocessing.get_l8_cloud_free_col(roi, ['NDVI','MNDWI','NDMI','SAVI'], 0.6)
predict_l8 = create_annual_composites_may_to_september(l8)
predict_l8 = filter_non_empty_bands(predict_l8)
predict_l8 = predict_l8.map(lambda image : normalize(add_elev_slope(image),roi))

display(predict_l8)

### Classification

In [68]:
def classification(image, classifier):
    img_classified = image.classify(classifier) 
    return image.addBands(img_classified)

In [69]:
l8Classified = predict_l8.map(lambda image : classification(image, classifier))
display(l8Classified)

In [28]:
img = landsat_preprocessing.get_l8_cloud_free_col_dates(roi, ['NDVI','MNDWI','NDMI','SAVI'], 0.6, '2023-05-01', '2023-09-30').median()

img = img.addBands(ee.Image(normalize(add_elev_slope(img),roi)).classify(classifier))

# Ajouter une légende à la carte
legend_dict = {
    'Northern Alluvial Meadows': '#91cf60',
    'Bare Land': '#fc8d59',
    'Water': '#4575b4',
    'Trees': '#1a9850',
    'Built': '#d73027',
}

mb = geemap.Map()

min_water_date = ee.Date('2022-09-25')
max_water_date = ee.Date('2018-05-09')

wetlands = wetlands_unsupervised_clustering.getWetlandsS2(roi, min_water_date, max_water_date)

wetlands_contours = wetlands.subtract(wetlands.focal_min(radius=2)).selfMask()

#left_layer = geemap.ee_tile_layer(img.select('classification').clip(roi), {'min': 0, 'max': 5, 'palette': ['#91cf60', '#fc8d59', '#4575b4', '#1a9850','#d73027','#fee08b']}, 'RF classification')
#right_layer = geemap.ee_tile_layer(img.clip(roi), {'min': 0, 'max': 1, 'bands': ['SR_B4', 'SR_B3', 'SR_B2']}, 'RGB color')
#mb.split_map(left_layer, right_layer)
mb.add_layer(apply_scale_factors(img.clip(roi)), {'min': 0, 'max': 0.1, 'bands': ['SR_B4', 'SR_B3', 'SR_B2']}, 'RGB color')
mb.add_layer(img.select('classification').clip(roi), {'min': 0, 'max': 5, 'palette': ['#91cf60', '#fc8d59', '#4575b4', '#1a9850','#d73027','#fee08b']}, 'RF classification')
mb.centerObject(roi, 14)
mb.add_legend(legend_title="Classification", legend_dict=legend_dict)
mb.addLayer(wetlands_contours,{'palette': ['blue']}, 'wetlands')

mb

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

## Surface time series 

### Count pixels for each classes

In [74]:
class_labels = [
    'svamangar', 'bare', 'water', 'trees', 'built'
]
def calculate_pixel_counts(image):
    pixel_count_stats = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram().unweighted(),
        geometry=roi,
        scale=30,
        maxPixels=1e10
    )
    pixel_counts = ee.Dictionary(pixel_count_stats.get('classification'))
    return ee.Feature(None,pixel_counts)

# Appliquer la fonction à chaque image de la collection
#pixel_counts_fc = ee.FeatureCollection(l8Classified.map(calculate_pixel_counts))
pixel_counts_fc = l8Classified.map(calculate_pixel_counts)
display(pixel_counts_fc)

### Transform into dataframe

In [85]:
classes_surfaces_df_raw = ee.data.computeFeatures({
    'expression': pixel_counts_fc,
    'fileFormat': 'PANDAS_DATAFRAME'
})

In [86]:
classes_surfaces_df_raw.head()

classes_surfaces_df = classes_surfaces_df_raw

In [87]:
# Création d'un dictionnaire inverse pour la légende
reverse_legend_dict = {
    '0': 'Northern Alluvial Meadows',
    '1': 'Bare Land',
    '2': 'Water',
    '3': 'Trees',
    '5': 'Built',
    'null': 'Crops',
    '4': 'Unknown'
}

# Remplacement des noms des classes par les étiquettes
classes_surfaces_df.rename(columns=reverse_legend_dict, inplace=True)

# Remplacement des surfaces en pixels par des hectares (en supposant 1 pixel = 1 hectare pour simplifier)
classes_surfaces_df= classes_surfaces_df.apply(pd.to_numeric, errors='ignore') * 0.09

classes_surfaces_df.drop(columns=['geo'], inplace=True)

# Ajout de la colonne 'year'
years = list(range(2023, 2023 - len(classes_surfaces_df), -1))
classes_surfaces_df['year'] = years

# Conversion de la dataframe pour une meilleure visualisation
df_melted = classes_surfaces_df.melt(id_vars=['year'], var_name='Class', value_name='Surface (hectares)')

# Création du graphique
fig = px.line(df_melted, x='year', y='Surface (hectares)', color='Class',
              title='Surface of different classes in Breforsen area derived from Landsat data yearly summer composite',
              labels={'year': 'Year', 'Surface (hectares)': 'Surface in Hectares'}, markers= True)

# Affichage du graphique
fig.show()

## Wetlands

In [89]:
min_water_date = ee.Date('2022-09-25')
max_water_date = ee.Date('2018-05-09')

wetlands = wetlands_unsupervised_clustering.getWetlandsS2(roi, min_water_date, max_water_date)

m = geemap.Map()
m.centerObject(roi,14)
m.addLayer(wetlands.selfMask(), {'palette': ['blue']}, 'mask')
m

def mask_with_wetlands(col,wetlands):
    col = col.map(lambda image : image.updateMask(wetlands))
    return col

l8Masked = mask_with_wetlands(l8Classified, wetlands)

display(l8Masked)

In [96]:
img = l8Masked.first()

# Ajouter une légende à la carte
legend_dict = {
    'Northern Alluvial Meadows': '#91cf60',
    'Bare Land': '#fc8d59',
    'Water': '#4575b4',
    'Trees': '#1a9850',
    'Built': '#d73027',
    'Crops': '#fee08b'
}

mb = geemap.Map()

#left_layer = geemap.ee_tile_layer(img.select('classification').clip(roi), {'min': 0, 'max': 5, 'palette': ['#91cf60', '#fc8d59', '#4575b4', '#1a9850','#d73027','#fee08b']}, 'RF classification')
#right_layer = geemap.ee_tile_layer(img.clip(roi), {'min': 0, 'max': 1, 'bands': ['SR_B4', 'SR_B3', 'SR_B2']}, 'RGB color')
#mb.split_map(left_layer, right_layer)
mb.addLayer(apply_scale_factors(training_img).clip(roi), {'bands': ['SR_B4', 'SR_B3', 'SR_B2'], 'min': 0, 'max': 0.1}, 'Sentinel-2 Image')
mb.add_layer(img.select('classification').clip(roi), {'min': 0, 'max': 5, 'palette': ['#91cf60', '#fc8d59', '#4575b4', '#1a9850','#d73027','#fee08b']}, 'RF classification')
mb.centerObject(roi, 14)
mb.add_legend(legend_title="Classification", legend_dict=legend_dict)
mb

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

In [92]:
class_labels = [
    'svamangar', 'bare', 'water', 'trees', 'built'
]
def calculate_pixel_counts(image):
    pixel_count_stats = image.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram().unweighted(),
        geometry=roi,
        scale=30,
        maxPixels=1e10
    )
    pixel_counts = ee.Dictionary(pixel_count_stats.get('classification'))
    return ee.Feature(None,pixel_counts)

# Appliquer la fonction à chaque image de la collection
#pixel_counts_fc = ee.FeatureCollection(l8Classified.map(calculate_pixel_counts))
pixel_counts_fc = l8Masked.map(calculate_pixel_counts)
display(pixel_counts_fc)

In [93]:
classes_surfaces_df_raw = ee.data.computeFeatures({
    'expression': pixel_counts_fc,
    'fileFormat': 'PANDAS_DATAFRAME'
})

classes_surfaces_df_raw.head()

classes_surfaces_df = classes_surfaces_df_raw

In [94]:
# Création d'un dictionnaire inverse pour la légende
reverse_legend_dict = {
    '0': 'Northern Alluvial Meadows',
    '1': 'Bare Land',
    '2': 'Water',
    '3': 'Trees',
    '5': 'Built',
    'null': 'Crops',
    '4': 'Unknown'
}

# Remplacement des noms des classes par les étiquettes
classes_surfaces_df.rename(columns=reverse_legend_dict, inplace=True)

# Remplacement des surfaces en pixels par des hectares (en supposant 1 pixel = 1 hectare pour simplifier)
classes_surfaces_df= classes_surfaces_df.apply(pd.to_numeric, errors='ignore') * 0.09

classes_surfaces_df.drop(columns=['geo'], inplace=True)

# Ajout de la colonne 'year'
years = list(range(2023, 2023 - len(classes_surfaces_df), -1))
classes_surfaces_df['year'] = years

# Conversion de la dataframe pour une meilleure visualisation
df_melted = classes_surfaces_df.melt(id_vars=['year'], var_name='Class', value_name='Surface (hectares)')

# Création du graphique
fig = px.line(df_melted, x='year', y='Surface (hectares)', color='Class',
              title='Surface of different classes in wetlands of Breforsen area derived from Landsat data yearly summer composite',
              labels={'year': 'Year', 'Surface (hectares)': 'Surface in Hectares'}, markers= True)

# Affichage du graphique
fig.show()